<a href="https://colab.research.google.com/github/African-Quant/WQU_MScFE_Capstone_Grp9/blob/master/rfc__model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook  --quiet
!pip install pyfolio --quiet

import fastbook


     |████████████████████████████████| 727kB 6.9MB/s 
     |████████████████████████████████| 194kB 8.6MB/s 
     |████████████████████████████████| 1.2MB 14.4MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 12.8MB 48.8MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
     |████████████████████████████████| 92kB 5.8MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 


In [2]:
import os
import re
  
import random
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import pandas as pd
from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
os.environ['PYTHONHASHSEED'] = '0'

import warnings
warnings.filterwarnings('ignore')

In [3]:
from fastbook import *

from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from pyfolio.timeseries import perf_stats 
from pyfolio import create_simple_tear_sheet, create_returns_tear_sheet

In [4]:
pairs = ['AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD', 'CAD', 'CADCHF', 
        'CADJPY', 'CHF', 'CHFJPY', 'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 
        'EURJPY', 'EURNZD', 'EURUSD', 'GBPAUD', 'GBPCAD', 'GBPCHF', 'GBPJPY', 
        'GBPNZD', 'GBPUSD', 'JPY', 'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD']

def get_data(pair):
        ''' Retrieves (from a github repo) and prepares the data.
        '''
        url = f'https://raw.githubusercontent.com/African-Quant/WQU_MScFE_Capstone_Grp9/master/Datasets/{pair}%3DX.csv'
        raw = pd.read_csv(url)
        raw = pd.DataFrame(raw).drop(['Adj Close', 'Volume'], axis=1)
        raw.iloc[:,0] = pd.to_datetime(raw.iloc[:,0])
        raw.set_index('Date', inplace=True)
        return raw

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path = '/content/drive/MyDrive/Capstone'

os.chdir(path)

In [7]:
from models import *

### Using  *Random Forest* to Predict Market Direction.

In [11]:
rfc_results_valid, rfc_results_test = rfc_results()

In [9]:
rfc_results_valid

pairs,Annual return,Cumulative returns,Annual volatility,Sharpe ratio,Calmar ratio,Stability,Max drawdown,Omega ratio,Sortino ratio,Skew,Kurtosis,Tail ratio,Daily value at risk
AUDCAD,-0.012469,-0.048572,0.084187,-0.106954,-0.054912,0.140157,-0.227071,0.982466,-0.147660,-0.191902,0.901709,1.000113,-0.010642
AUDCHF,-0.075711,-0.268327,0.135731,-0.509593,-0.227200,0.864907,-0.333235,0.904300,-0.614677,-5.828005,102.079169,0.843804,-0.017375
AUDJPY,0.013083,0.052935,0.126366,0.166110,0.055301,0.399555,-0.236585,1.028654,0.228432,-0.356183,1.985302,0.938941,-0.015837
AUDNZD,0.017777,0.072428,0.078118,0.264631,0.147192,0.197047,-0.120777,1.045482,0.367418,-0.259972,1.712467,0.921325,-0.009760
AUDUSD,-0.069280,-0.247918,0.104573,-0.634186,-0.248604,0.836225,-0.278678,0.902293,-0.846608,-0.186355,0.253400,0.871438,-0.013438
CAD,0.003635,0.014502,0.077244,0.085528,0.027209,0.126543,-0.133593,1.014268,0.125240,0.225680,0.984775,1.044500,-0.009706
CADCHF,-0.061398,-0.222324,0.126493,-0.434468,-0.199461,0.662266,-0.307822,0.910744,-0.519328,-8.356579,170.463812,1.041961,-0.016155
CADJPY,0.041166,0.173609,0.117312,0.402510,0.305251,0.238219,-0.134859,1.072226,0.579029,-0.026430,2.908339,1.017492,-0.014592
CHF,-0.057465,-0.209311,0.120531,-0.427460,-0.184579,0.764788,-0.311330,0.909291,-0.505668,-9.412203,202.548659,0.923586,-0.015390
CHFJPY,-0.052745,-0.193482,0.132029,-0.340217,-0.168635,0.406223,-0.312779,0.925152,-0.400266,-9.884841,217.443498,0.940102,-0.016812


In [ ]:
rfc_results_test

pairs,Annual return,Cumulative returns,Annual volatility,Sharpe ratio,Calmar ratio,Stability,Max drawdown,Omega ratio,Sortino ratio,Skew,Kurtosis,Tail ratio,Daily value at risk
AUDCAD,-0.023944,-0.091692,0.072356,-0.298739,-0.081714,0.670248,-0.293023,0.950166,-0.408385,-0.375870,3.647413,1.018495,-0.009202
AUDCHF,-0.016129,-0.062487,0.092672,-0.129138,-0.052202,0.707204,-0.308968,0.978847,-0.179873,-0.112541,1.267715,0.940482,-0.011723
AUDJPY,0.034689,0.144906,0.108252,0.369120,0.204849,0.248790,-0.169341,1.065672,0.528905,-0.049909,2.659303,1.054888,-0.013480
AUDNZD,-0.005575,-0.021941,0.055850,-0.072187,-0.063941,0.086044,-0.087193,0.988111,-0.100147,-0.250940,2.083767,1.027484,-0.007052
AUDUSD,-0.038069,-0.142743,0.094926,-0.361364,-0.110231,0.762562,-0.345356,0.939902,-0.493810,-0.247119,2.699930,0.945024,-0.012096
CAD,0.034668,0.144814,0.073016,0.503187,0.203744,0.000719,-0.170156,1.092657,0.773188,0.637964,4.916182,1.124168,-0.009053
CADCHF,-0.042283,-0.157550,0.080095,-0.499283,-0.230089,0.738217,-0.183769,0.920389,-0.674796,-0.406059,2.815421,0.985926,-0.010250
CADJPY,-0.037371,-0.140272,0.095637,-0.350415,-0.111084,0.813953,-0.336420,0.942587,-0.486275,-0.090443,2.720504,0.971875,-0.012182
CHF,-0.008698,-0.034072,0.066811,-0.097357,-0.039779,0.112897,-0.218651,0.983825,-0.135322,-0.198739,1.992724,1.002160,-0.008443
CHFJPY,0.002568,0.010228,0.063705,0.072058,0.024430,0.009587,-0.105101,1.012180,0.103665,0.202681,2.920751,1.028668,-0.008008
